In [14]:
import netket as nk
from netket.operator.spin import sigmax,sigmaz, sigmap, sigmam, sigmay
from scipy.sparse.linalg import eigsh
import jax.numpy as jnp

N = 16
hi = nk.hilbert.Spin(s=1 / 2, N=N)
theta1 = 0.25+0.0j
theta2 = 0.25+0.0j
H = 2*sum([sigmax(hi, i)*sigmax(hi, i+1)+sigmay(hi, i)*sigmay(hi, i+1) for i in range(1, int(N/2))])
H += 2*sum([sigmam(hi, i)*sigmap(hi, (i+1)%N)+sigmap(hi, i)*sigmam(hi, (i+1)%N) for i in range(int(N/2+1), N)])
H += 2*jnp.cos(theta1)*(sigmax(hi, 0)*sigmax(hi, 1)+sigmay(hi, 0)*sigmay(hi, 1))+jnp.sin(theta1)*(sigmax(hi, 0)*sigmay(hi, 1)-sigmay(hi, 0)*sigmax(hi, 1))
H += 2*jnp.cos(theta2)*(sigmax(hi, int(N/2))*sigmax(hi, int(N/2)+1)+sigmay(hi, int(N/2))*sigmay(hi, int(N/2)+1))+jnp.sin(theta2)*(sigmax(hi, int(N/2))*sigmay(hi, int(N/2)+1)-sigmay(hi, int(N/2))*sigmax(hi, int(N/2)+1))
H += sum([sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])
sp_h=H.to_sparse()
eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")

print("eigenvalues with scipy sparse:", eig_vals)

E_gs = eig_vals[0]

eigenvalues with scipy sparse: [-39.68304359 -38.43108525]


In [1]:
import netket as nk
from netket.operator.spin import sigmax,sigmaz, sigmap, sigmam, sigmay
from scipy.sparse.linalg import eigsh
import jax.numpy as jnp


In [33]:
import netket as nk

# Define the Hilbert space, typically for a spin system
hilbert_space = nk.hilbert.Spin(s=0.5, N=1)  # Example for a single spin-1/2 system

# Define the σ^x and σ^z operators
sigmax = nk.operator.spin.sigmax(hilbert=hilbert_space, site=0)
sigmaz = nk.operator.spin.sigmaz(hilbert=hilbert_space, site=0)

# Convert operators to dense matrix form to access elements
matrix_sigmax = (identity(hilbert_space)-sigmax).to_dense()
matrix_sigmaz = sigmaz.to_dense()
print(matrix_sigmax)

[[ 1. -1.]
 [-1.  1.]]


In [60]:
import jax
import netket as nk
import numpy as np
from netket.operator.spin import sigmax,sigmaz, sigmap, sigmam, identity
import time
from scipy.sparse.linalg import eigsh
import jax.numpy as jnp
import matplotlib.pyplot as plt
from twoD_tool import *
Lx = 4
Ly = 3
N = Lx * Ly
params = [ 0.4, 0.8, 1.2, 1.6, 2.0, 2.4, 2.8, 3.2, 3.6, 4.0, 4.4 ]
Omega = 1.0
Rb = 1.2**6
hi = nk.hilbert.Spin(s=1 / 2, N =  N)
for param in params:
    ni = []
    H = Omega/2*sum([sigmax(hi, y*Lx+x) for y in range (Ly) for x in range (Lx)]) #X
    H -= param/2*sum([(identity(hi)-sigmaz(hi, y*Lx+x)) for y in range(Ly) for x in range(Lx)])
    H += Omega*Rb/4*sum([((identity(hi)-sigmaz(hi, y1*Lx+x1))*(identity(hi)-sigmaz(hi, y1*Lx+x2)))/((x1-x2)**2)**3 \
                         for y1 in range(Ly) for x1 in range(Lx) for x2 in range(x1+1, Lx)])
    H += Omega*Rb/4*sum([((identity(hi)-sigmaz(hi, y1*Lx+x1))*(identity(hi)-sigmaz(hi, y2*Lx+x2)))/(((x1-x2)**2+(y1-y2)**2)**3) \
                         for y1 in range(Ly) for x1 in range(Lx) for y2 in range(y1+1, Ly) for x2 in range(Lx)])
    sp_h = H.to_sparse()
    eig_vals, eig_vecs = eigsh(sp_h, k=3, which="SA")
    
    stagger_H = sum([sigmaz(hi, y*Lx+x)*(-1)**(y*Lx+x) for y in range(Ly) for x in range (Lx)])
    '''
    for y in range(Ly):
        for x in range(Lx):
            h_ni = (identity(hi)-sigmaz(hi, y*Lx+x))/2
            ni.append(eig_vecs[:,0].conj() @ h_ni.to_sparse() @ eig_vecs[:,0])
    '''
    stagger_H_sp = stagger_H.to_sparse()
    stagger_mag0 = np.abs(eig_vecs[:,0].conj() @ stagger_H_sp @ eig_vecs[:,0])
    stagger_mag1 = np.abs(eig_vecs[:,0].conj() @ stagger_H_sp @ eig_vecs[:,1])
    stagger_mag2 = np.abs(eig_vecs[:,1].conj() @ stagger_H_sp @ eig_vecs[:,1])
    stagger_mag3 = np.abs(eig_vecs[:,0].conj() @ stagger_H_sp @ eig_vecs[:,2])
    stagger_mag4 = np.abs(eig_vecs[:,1].conj() @ stagger_H_sp @ eig_vecs[:,2])
    stagger_mag5 = np.abs(eig_vecs[:,2].conj() @ stagger_H_sp @ eig_vecs[:,2])
    print(stagger_mag0, stagger_mag1, stagger_mag2, stagger_mag3, stagger_mag4, stagger_mag5)
    print(eig_vals[1]-eig_vals[0])
    

1.8207657603852567e-14 2.042899866990031 7.549516567451064e-15 3.1183389204159084e-14 0.3263719804474074 3.835820550079916e-14
0.5489903306276469
4.440892098500626e-15 2.257442460264978 2.220446049250313e-15 2.88935542158697e-14 1.5836217676767925 1.9539925233402755e-14
0.38150904794186946
7.438494264988549e-15 2.3470211534155663 1.212363542890671e-13 5.362377208939506e-14 2.5258885408528284 1.1124434706744069e-13
0.24842219673215737
4.1244785364824565e-13 2.6338595472318547 2.935429677108914e-13 1.4155343563970746e-13 2.609189453235979 1.141309269314661e-13
0.13072486762790625
7.707168236947837e-13 3.310764818375614 6.641354133307686e-13 1.1379786002407855e-13 1.7976267665989467 1.0491607582707729e-13
0.043398902090654445
2.2164492463616625e-12 3.647289574424381 2.255973186038318e-12 3.5599301284605644e-13 0.9484951791282564 4.068967385251199e-14
0.011776876367243005
3.128164394183841e-12 3.708286148259544 3.1437075165285933e-12 2.9071189899809724e-13 0.597555662982868 1.0103029524088

In [54]:
hi = nk.hilbert.Spin(s=1 / 2, N =  1)
H = identity(hi)-sigmaz(hi, 0)
H.to_sparse() @ [1, 0]

array([0., 0.])

(512, 1)